In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import os
import re
import numpy as np
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")
import warnings
warnings.filterwarnings('ignore')

In [ ]:
browser= webdriver.Chrome('C:\Selenium\chromedriver', chrome_options=options)
url=r'https://www.topuniversities.com/university-rankings/world-university-rankings/2019'
browser.get(url)

In [ ]:
browser.execute_script("window.scrollTo(0, 250)") 
hover_elem = browser.find_element_by_xpath('//*[@id="cid-410"]/td[2]/div/a[1]')
hover = ActionChains(browser).move_to_element(hover_elem)
hover.perform() 

In [ ]:
drop=browser.find_element_by_xpath('//*[@id="qs-rankings"]/thead/tr[3]/td[3]/div/div/span[2]/span[2]')
hover = ActionChains(browser).move_to_element(drop)
hover.perform()
drop.click() 

In [ ]:
#list of items in the drop down menu
a=browser.find_elements_by_class_name('jcf-option')

In [ ]:
scroll = browser.find_element_by_xpath('//*[@id="qs-rankings_length"]/label/span[2]/span[2]')
element_to_hover_over = scroll
hover_scroll = ActionChains(browser).move_to_element(element_to_hover_over)
hover_scroll.perform()
scroll.click()

In [ ]:
def input_region(region):
    for i in range(len(a)):
        if i<7 and a[i].text==region:
            print(a[i].text)
            a[i].click()
            break

input_region('Asia')

In [ ]:
#Standard Function for search

# def uni_search_by_name(uni_name):
#     uni_search=browser.find_element_by_xpath('//*[@id="qs-rankings"]/thead/tr[3]/td[2]/div/input')
#     uni_name=input('Enter_uni_name: ')
#     uni_search.send_keys(uni_name)
#     #Range of other functions doing their stuff here
    
    

In [ ]:
#get all the results in a page

scr_all_menu=browser.find_element_by_xpath('//*[@id="qs-rankings_length"]/label/span[2]/span[2]')
hover_scroll = ActionChains(browser).move_to_element(scr_all_menu)
hover_scroll.perform()
scr_all_menu.click()

scr_all=browser.find_element_by_xpath('//*[@id="qs-rankings_length"]/label/span[2]/div/div/span/span/ul/li[5]/span')
hover_scroll = ActionChains(browser).move_to_element(scr_all)
hover_scroll.perform()
scr_all.click()


In [ ]:
#getting the list of universities
def get_uni():
    get_uni.list_uni=browser.find_elements_by_class_name('title')
    uni=[]
    for i in range(len(get_uni.list_uni)):
        uni.append(get_uni.list_uni[i].text)
    return uni
uni= get_uni()


In [ ]:
#getting the ranks of universities
def get_rank():
    list_rank=browser.find_elements_by_class_name('rank')
    rank_list=[]
    for i in range(int(len(list_rank)/2)):
        if i>0:
            if list_rank[i*2+1].text!='':
                rank_list.append(list_rank[i*2+1].text)
#                 if bool(re.search('=',list_rank[i*2+1].text)):
#                     rank_list.append(re.findall('\d{1,5}',list_rank[i*2+1].text)
#                 else :       
            else:
                break
    return rank_list
rank = get_rank()

In [ ]:
#finding the country 
# k=1
def get_country():
    list_cntry=browser.find_elements_by_class_name('td-wrap')
    ctry_list=[]
    for i in range(int((len(list_cntry)-15)/5)):
        j=i*5+15+2
        if list_cntry[j].text!='':
            ctry_list.append(list_cntry[j].text)
    return ctry_list
ctry = get_country()


In [ ]:
def uni_data(ctry,rank,uni):
    df_ctry = pd.DataFrame(ctry,columns=['Country'])
    df_rank = pd.DataFrame(rank,columns=['Rank'])
    df_uni = pd.DataFrame(uni,columns=['University'])
    df=pd.concat([df_rank,df_ctry,df_uni],axis=1)
    return df

In [ ]:
def uni_info(rank):
    get_uni.list_uni[rank+1].click()

In [ ]:
#to fetch all para
def fetch_para():
    browser.find_element_by_class_name('more-link').click()
    para= browser.find_elements_by_class_name('field-profile-overview')
    list_paras = para[1].find_elements_by_tag_name('p')
    para_str=''
    for i in range(len(list_paras)-1):
        para_str+=list_paras[i].text
    return para_str

In [ ]:
def details():
    val = browser.find_elements_by_class_name('val')
    lab = browser.find_elements_by_class_name('lab')
    lab_list=[]
    val_list=[]
    for i in range(len(val)-1):
        val_list.append(val[i].text)
        lab_list.append(lab[i].text)
    return pd.DataFrame.from_dict(data=dict(zip(lab_list,val_list)),orient='index').T
details_df=details()

In [ ]:
def location():
    locality = browser.find_element_by_class_name('locality').text
    state = browser.find_element_by_class_name('state').text
    pin = browser.find_element_by_class_name('postal-code').text
    country = browser.find_elements_by_class_name('country')[1].text
    return pd.DataFrame.from_dict({'locality':locality,'State':state,'PIN':pin,"Country":country},orient='index').T
loc_df =location()

In [ ]:
def com_details():
    return pd.concat([details(),location()],axis=1)
df=com_details()
df

In [ ]:
def google_the_link(uni_name):
    browser.execute_script('''window.open("http://google.com","_blank");''')
    #windows_before  = browser.current_window_handle
    #print("First Window Handle is : %s" %windows_before)
    browser.switch_to_window(browser.window_handles[-1])
    search_name=WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.NAME, 'q')))
    search_name.send_keys(uni_name)
    browser.find_element_by_id('lga').click()
    browser.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[3]/center/input[1]').click()
    WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'q')))
    sag=browser.find_elements_by_class_name('r')
    #link (First Search result)
    link = sag[0].find_element_by_tag_name('a').get_attribute('href')
    return link
    


In [ ]:
df['Link']=google_the_link(uni_name)